## 1. Import Libraries

First, we need to import the necessary libraries.

In [9]:
import os
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import json


In [10]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

tf.config.run_functions_eagerly(True)

1 Physical GPUs, 1 Logical GPU


## 2. Define Directories

Next, we define the directories for the processed data and the saved models.

In [11]:
BASE_DIR = r"G:\OneDrive\ML-MinorProject\ISIC-2020 Dataset"
PROCESSED_TRAIN_DIR = os.path.join(BASE_DIR, "processed_train")
PROCESSED_VAL_DIR = os.path.join(BASE_DIR, "processed_val")
PROCESSED_TEST_DIR = os.path.join(BASE_DIR, "processed_test")

MODEL_DIR = r"G:\OneDrive\ML-MinorProject\models"
MOBILENET_MODEL_PATH = os.path.join(MODEL_DIR, "mobilenet", "mobilenet_best_model.h5")

## 3. Image Processing Configurations and Data Augmentation

We define the image processing configurations and create data generators.

In [12]:
# Adjusted hyperparameters for MobileNetV2
IMG_SIZE = (224, 224)  # MobileNetV2 input size
BATCH_SIZE = 16  # Increased batch size due to smaller input size

# Data augmentation and preprocessing configurations
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Preprocessing specific to MobileNetV2, normalized pixel values ranges bw [-1,1]  
    rotation_range=20,                        # Reduced rotation range as mobilenetV2 is lightweight in nature
    width_shift_range=0.1,                    # Reduced shift ranges (similar reasons)
    height_shift_range=0.1,
    shear_range=0.1,                          # Reduced shear
    zoom_range=0.1,                           # Reduced zoom
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    PROCESSED_TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True
)

# Validation data generator
val_generator = val_datagen.flow_from_directory(
    PROCESSED_VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# Testing data generator
test_generator = test_datagen.flow_from_directory(
    PROCESSED_TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)


Found 26500 images belonging to 2 classes.
Found 6626 images belonging to 2 classes.
Found 10982 images belonging to 2 classes.


## 4. Visualize Data Augmentation

We visualize some augmented images to ensure that the augmentations are applied correctly.

In [ ]:
def visualize_augmentation(generator):
    batch = next(generator)
    images = batch[0]  # Get the batch of images (input data)

    # Denormalize images to [0, 1] range for visualization
    images = (images + 1) / 2.0  # Rescale from [-1, 1] to [0, 1]

    plt.figure(figsize=(15, 5))
    for i in range(7):
        plt.subplot(1, 7, i + 1)
        plt.imshow(images[i])  # Display the image
        plt.axis('off')
    plt.suptitle("Augmented Images")
    plt.show()

# Call the function to visualize augmented images
visualize_augmentation(train_generator)

## 5. Enable Mixed Precision

We enable mixed precision to improve training performance.

In [13]:
from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

## 6. Define and Compile Model

We define and compile the MobileNetV2 model architecture.

In [14]:
# Define the base model for MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Build the full model
model = Sequential([
    base_model,                             # MobileNetV2 as the base model
    GlobalAveragePooling2D(),               # Pooling layer
    Dropout(0.5),                           # Regularization with dropout
    Dense(1, activation='sigmoid', dtype='float32')  # Binary classification
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),   # Adam optimizer with a lower learning rate
    loss='binary_crossentropy',            # Loss function for binary classification
    metrics=['accuracy']                   # Metric to track during training
)

# Display the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_1 (Dropout)         (None, 1280)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1281      
                                                                 
Total params: 2,259,265
Trainable params: 2,225,153
Non-trainable params: 34,112
_________________________________________________________________


## 7. Define Callbacks

We define callbacks to improve the training process.

In [15]:
# 1. Save the best model during training
checkpoint = ModelCheckpoint(
    filepath='mobilenetv2_best_model.h5',  # Save the model to this file
    monitor='val_loss',                   # Monitor validation loss
    save_best_only=True,                  # Save only the best model
    save_weights_only=False,              # Save the entire model
    mode='min',                           # Look for the minimum val_loss
    verbose=1                             # Print saving progress
)

# 2. Stop training early if validation loss stops improving
early_stopping = EarlyStopping(
    monitor='val_loss',                   # Monitor validation loss
    patience=10,                          # Stop after 10 epochs of no improvement
    mode='min',                           # Look for the minimum val_loss
    restore_best_weights=True,            # Restore the best weights at the end
    verbose=1                             # Print early stopping message
)

# 3. Reduce learning rate when validation loss plateaus
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',                   # Monitor validation loss
    factor=0.2,                           # Reduce the learning rate by 20%
    patience=5,                           # Wait for 5 epochs before reducing
    min_lr=1e-6,                          # Minimum learning rate
    mode='min',                           # Look for the minimum val_loss
    verbose=1                             # Print learning rate reduction message
)

# Combine the callbacks
callbacks = [checkpoint, early_stopping, reduce_lr]

## 8. Train the Model

We train the model using the data generators.

In [16]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=callbacks
)

Epoch 1/20
1657/1657 [==============================] - ETA: 0s - loss: 0.0842 - accuracy: 0.9806
Epoch 1: val_loss improved from inf to 0.11062, saving model to mobilenetv2_best_model.h5
1657/1657 [==============================] - 2729s 2s/step - loss: 0.0842 - accuracy: 0.9806 - val_loss: 0.1106 - val_accuracy: 0.9823 - lr: 1.0000e-04
Epoch 2/20
1657/1657 [==============================] - ETA: 0s - loss: 0.0738 - accuracy: 0.9824
Epoch 2: val_loss did not improve from 0.11062
1657/1657 [==============================] - 2719s 2s/step - loss: 0.0738 - accuracy: 0.9824 - val_loss: 0.1281 - val_accuracy: 0.9823 - lr: 1.0000e-04
Epoch 3/20
1657/1657 [==============================] - ETA: 0s - loss: 0.0696 - accuracy: 0.9824
Epoch 3: val_loss improved from 0.11062 to 0.08151, saving model to mobilenetv2_best_model.h5
1657/1657 [==============================] - 2693s 2s/step - loss: 0.0696 - accuracy: 0.9824 - val_loss: 0.0815 - val_accuracy: 0.9823 - lr: 1.0000e-04
Epoch 4/20
1657/165

## 9. Fine-Tune the Model

We unfreeze the base model and fine-tune the entire model.

In [17]:
# Unfreeze the base model
base_model.trainable = True

# Recompile the model with a lower learning rate
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
fine_tune_history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=callbacks
)



Epoch 1/10
1657/1657 [==============================] - ETA: 0s - loss: 0.0606 - accuracy: 0.9824
Epoch 1: val_loss improved from 0.08151 to 0.07638, saving model to mobilenetv2_best_model.h5
1657/1657 [==============================] - 2499s 2s/step - loss: 0.0606 - accuracy: 0.9824 - val_loss: 0.0764 - val_accuracy: 0.9822 - lr: 1.0000e-05
Epoch 2/10
1657/1657 [==============================] - ETA: 0s - loss: 0.0572 - accuracy: 0.9825
Epoch 2: val_loss did not improve from 0.07638
1657/1657 [==============================] - 2311s 1s/step - loss: 0.0572 - accuracy: 0.9825 - val_loss: 0.0786 - val_accuracy: 0.9823 - lr: 1.0000e-05
Epoch 3/10
1657/1657 [==============================] - ETA: 0s - loss: 0.0553 - accuracy: 0.9824
Epoch 3: val_loss improved from 0.07638 to 0.07442, saving model to mobilenetv2_best_model.h5
1657/1657 [==============================] - 2327s 1s/step - loss: 0.0553 - accuracy: 0.9824 - val_loss: 0.0744 - val_accuracy: 0.9825 - lr: 1.0000e-05
Epoch 4/10
1657

## 10. Save the Final Model

We save the final model after training and fine-tuning.

In [18]:
final_model_path = os.path.join(MODEL_DIR, "mobilenet", "mobilenet_final_model.h5")
model.save(final_model_path)
print(f"Final model saved to {final_model_path}")

Final model saved to G:\OneDrive\ML-MinorProject\models\mobilenet\mobilenet_final_model.h5
